In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"s303kd","key":"e16a0e046f9deb5afbb2070827e344ec"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d fanconic/skin-cancer-malignant-vs-benign
#https://www.kaggle.com/datasets/fanconic/skin-cancer-malignant-vs-benign

 99% 323M/325M [00:02<00:00, 194MB/s]
100% 325M/325M [00:02<00:00, 162MB/s]


In [ ]:
!unzip -q skin-cancer-malignant-vs-benign.zip

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import  Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/content/train'
test_dir = '/content/test'

In [ ]:
train_benign = len(os.listdir(os.path.join(train_dir, 'benign')))
train_malignant = len(os.listdir(os.path.join(train_dir, 'malignant')))
test_benign = len(os.listdir(os.path.join(test_dir, 'benign')))
test_malignant = len(os.listdir(os.path.join(test_dir, 'malignant')))

In [ ]:
# Printing the counts
print("Train set:")
print("Benign:", train_benign)
print("Malignant:", train_malignant)
print("\nTest set:")
print("Benign:", test_benign)
print("Malignant:", test_malignant)

Train set:
Benign: 1440
Malignant: 1197

Test set:
Benign: 360
Malignant: 300


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21,137,729
Trainable params: 21,137,729
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 2637 images belonging to 2 classes.


In [ ]:
#Limit the number of samples to 70 per class
class_counts = train_generator.classes
class_labels = list(train_generator.class_indices.keys())
for label in class_labels:
    indices = [i for i, cls in enumerate(class_counts) if cls == train_generator.class_indices[label]]
    if len(indices) > 70:
        train_generator.index_array = indices[:70]
        train_generator.samples = 70

# Check the number of samples after limiting
num_train_samples = train_generator.samples
print("Number of training samples after limiting:", num_train_samples)

Number of training samples after limiting: 70


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 660 images belonging to 2 classes.


In [ ]:
# Limit the number of samples to 25 per class
class_counts = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
for label in class_labels:
    indices = [i for i, cls in enumerate(class_counts) if cls == test_generator.class_indices[label]]
    if len(indices) > 25:
        test_generator.index_array = indices[:25]
        test_generator.samples = 25

# Check the number of samples after limiting
num_test_samples = test_generator.samples
print("Number of testing samples after limiting:", num_test_samples)

Number of testing samples after limiting: 25


In [ ]:
model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10
83/83 [==============================] - 182s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
48/83 [================>.............] - ETA: 40:54 - loss: 0.5423 - accuracy: 0.4577